In [2]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [5]:
# Alternative method using chatbot library 
import chatterbot
from chatterbot import Chatbot
from chatterbot.trainer import ListTrainer

# learning based on previous dialog to generate responses 

ModuleNotFoundError: No module named 'chatterbot'

In [9]:
with open('Users/riya/Desktop/Chatbot/intents.json') as file:
    data = json.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'Users/riya/Desktop/Chatbot/intents.json'

In [1]:
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

print("TS", training_sentences, "\n")
print(training_labels, "\n")
print(labels, "\n")
print(responses, "\n")

NameError: name 'data' is not defined

In [ ]:
# Tokenizing Process

# first need to read through the intents json file
# then we tokenize the words ex" "words" --> "w", "o", "r", "d", "s"
# stemming/ lemitization i.e we strip the words to its root word 
# Bag of Words: store the frequency of each word 
# vector example: [0,1,1,1,0,1]

In [ ]:
# Different ways of going about it, labelEncoder helps to convert data into numbers model will understand

# Using Tensorflow/Keras libraries (denseLayer) 
# Alternative:
# Or can use Tflearn model (fully_connected())

In [6]:
from sklearn.preprocessing import LabelEncoder

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels) 
training_labels = lbl_encoder.transform(training_labels) # converts into data model can understand 

training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1])

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token) # adding out of vocabulary token
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [1]:
import pickle
from tensorflow import keras
from tensorflow.keras.models import Sequential

model = Sequential() # stack like model
# Documentation:
# tf.keras.layers.Embedding(input_dim, output_dim, input_length)
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len)) # search this too!

# converts 3D vector to 2D vector
model.add(GlobalAveragePooling1D())

# relu acitvation function (usually default) in hidden layer to avoid vanishing gradient problem
# non-linear activation function: advanatge is that it does not activate all the neurons at once
model.add(Dense(16, activation='relu'))  
model.add(Dense(16, activation='relu'))
# softmax converts raw output into a vector of probabilities 
model.add(Dense(num_classes, activation='softmax')) # search what softmax does !

model.compile(loss='sparse_categorical_crossentropy',   
              optimizer='adam', metrics=['accuracy'])

# print("here")
print(model.summary())

In [ ]:
# epochs = 550
history = model.fit(padded_sequences, np.array(training_labels), epochs=550)

In [ ]:
# to save the trained model
# lets go through this 
model.save("chat_model")

# pickle used to serialize/ deserialize objects 
# pickling is a way to convert a python object into a character stream


# to save the fitted tokenizer, so we dont have to do this all over again!
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)